In [21]:
import pandas as pd

In [22]:
labels = pd.read_csv('art500k/label_list.csv', delimiter='|')

head_info_csv = pd.read_csv('art500k/head_info.csv')  # Adjust file name if necessary
new_column_names = list(head_info_csv.columns)  # Extract column names

#labels.columns = new_column_names


print(labels.head())


/var/folders/37/6l880pbs70bgr73m4qbsq0080000gn/T/ipykernel_11497/2124946217.py:1: DtypeWarning: Columns (3,5,6,7,10,12,13,15,17,19,20,21,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  labels = pd.read_csv('art500k/label_list.csv', delimiter='|')


  Gustave Courbet                Woman With A Parrot##AAHozJAL0gqXcA  \
0   Auguste Rodin         La Tentation Saint Antoine##WAGC82imJTDyIg   
1     Frida Kahlo   Retrato De Alejandro Gómez Arias##0QFuguLe4xyN_A   
2          Banksy           The Wall Banksy Balloons##FgHoVE-hmt6DBQ   
3        El Greco                     The Visitation##HQEQ_qXDtRrzkA   
4        El Greco  Madonna And Child With Saint Martina And Saint...   

  http://lh3.googleusercontent.com/nPuY2uKye4csy5XdoAjLo9BBL-vz7MKcBLZPuqRFJGai6bDgzoiaXVYa-E0JIzELwQ  \
0  http://lh3.googleusercontent.com/CgGN-xuGOgzBu...                                                    
1  http://lh3.googleusercontent.com/j-stBr_Q1mblB...                                                    
2  http://lh4.ggpht.com/DajPMSanJJDk7gPghfEqoT5Ru...                                                    
3  http://lh4.ggpht.com/dYGuyZERVymx98N9Uw3KKvOU6...                                                    
4  http://lh3.googleusercontent.com/jw0pUX

In [23]:
year_labels = labels
year_labels.columns = list(range(year_labels.shape[1] - 1)) + ["path"]
year_labels = labels[labels[9].notna()]
columns_to_keep = [9, 'path']
year_labels_filtered = year_labels[columns_to_keep]
year_labels_filtered = year_labels_filtered.rename(columns={9: 'date'})
#style_labels_filtered = style_labels_filtered[style_labels_filtered['path'].notna()]

In [24]:
import re

def extract_century(date_str):
    # lower-case date strings
    date_str = date_str.lower()

    # handling 'century' formatting (bc, bce, etc.)
    century_match = re.search(r"(?:early|late)?\s*(\d{1,2})(?:st|nd|rd|th) century\s*(bc|bce|b\.c\.|b\.c\.e\.)?", date_str)
    if century_match:
        century = int(century_match.group(1))
        if century_match.group(2):  # If BCE variation is present
            return -century
        return century

    # handle plural formatting
    decade_match = re.search(r"(\d{2})00s", date_str)
    if decade_match:
        return int(decade_match.group(1))

    # extract 4 digit years from string
    years = re.findall(r"\b\d{4}\b", date_str)

    if not years:
        # handle bc
        bce_year_match = re.findall(r"(\d{1,4})\s*(bc|bce|b\.c\.|b\.c\.e\.)", date_str)
        if bce_year_match:
            earliest_year = min(int(y[0]) for y in bce_year_match)  # Take the earliest year in a range
            century = (earliest_year - 1) // 100 + 1
            return -century  # Negative century for BCE

        # handle date ranges with bc
        bce_range_match = re.search(r"(\d{1,4})\s*-\s*(\d{1,4})\s*(bc|bce|b\.c\.|b\.c\.e\.)", date_str)
        if bce_range_match:
            earliest_year = min(int(bce_range_match.group(1)), int(bce_range_match.group(2)))
            century = (earliest_year - 1) // 100 + 1
            return -century  # Negative century for BCE

        return None  # return none if no year found

    # convert first year to century
    first_year = int(years[0])
    century = (first_year - 1) // 100 + 1
    return century



# apply funciton on our dates column
year_labels_filtered["century"] = year_labels_filtered["date"].apply(extract_century)


#since were still missing some bc handling despite my best efforts, we will just add a manual fix for the bc values
year_labels_filtered["century"] = year_labels_filtered["century"].apply(lambda x: -x+1 if x is not None and x > 21 else x)



#classifying the art by century buckets

# defining bucets
century_mapping = {
    "Ancient": range(-59, 10),   # Anything before the 10th century
    "Medieval": range(10, 15),   # 10th to 14th century
    "Renaissance": range(15, 17),  # 15th to 16th century
    "Baroque": range(17, 18),    # 17th century
    "Enlightenment": range(18, 19),  # 18th century
    "Modern": range(19, 21),     # 19th to 20th century
    "Contemporary": range(21, 22)  # 21st century
}

# assigning numeric label to each bucket
period_labels = {name: idx for idx, name in enumerate(century_mapping.keys())}


def map_century_to_period(century):
    try:
        century = int(century)  # Convert safely to int
        for period, centuries in century_mapping.items():
            if century in centuries:
                return period_labels[period]
    except (ValueError, TypeError):  # Catch NaN or unexpected values
        return None  # Return None if mapping fails
    return None  # Return None for unmapped centuries

# apply to mapping
year_labels_filtered["Period_Label"] = year_labels_filtered["century"].apply(map_century_to_period)

# drop rows where mapping failed 
year_labels_filtered = year_labels_filtered.dropna(subset=["Period_Label"]).reset_index(drop=True)

year_labels_filtered = year_labels_filtered[year_labels_filtered['path'].notna()]

columns_to_keep = ['path', 'Period_Label']

year_labels_filtered = year_labels_filtered[columns_to_keep]
year_labels_filtered = year_labels_filtered[year_labels_filtered["Period_Label"] != 5.0]
year_labels_filtered

,path,Period_Label
0,Artists1/El Greco/The Visitation##HQEQ_qXDtRrz...,3.0
1,Artists1/Gim Hongdo/Ducks In Winter##lAHeyXhD5...,4.0
7,Artists1/Jean-Baptiste-Siméon Chardin/Still Li...,4.0
14520,Artists1/Utagawa Toyokuni/Sumo E Ukiyo E Print...,4.0
14533,Artists1/Fra Angelico/Annunciatory Angel##TwGE...,2.0
...,...,...
242309,Artists1/Canvas/Harvesting Rice##vQFqHsR4cuMki...,6.0
242310,Artists1/Canvas/Virgin Bust##FAEestgT1xLQXA.jpg,4.0
242311,Artists1/Canvas/Mali Genest « Où Va Le Blanc ...,6.0
242312,Artists1/Canvas/Family Portrait##vAE7h7dtFUhJm...,4.0


In [25]:
# Load the new dataset
new_data = pd.read_csv('art500k/artwork_dataset.csv')
new_data["century"] = new_data["picture data"].apply(extract_century)
# Apply the function to map centuries to period labels
new_data["Period_Label"] = new_data["century"].apply(map_century_to_period)

# Drop rows where mapping failed
new_data = new_data.dropna(subset=["Period_Label"]).reset_index(drop=True)

# Display the first few rows to verify

columns_to_keep_new = ['ID', 'Period_Label']
new_data = new_data[columns_to_keep_new]

# Construct the new DataFrame
new_data_combined = pd.DataFrame({
    "path": "artwork/" + new_data ["ID"].astype(str) + ".jpg", # Construct the path
    "Period_Label": new_data ["Period_Label"] # Use the mapped period labels
})
# Disolay the first few rows to verify
print(new_data_combined.head())

# Load your existing dataset
# Replace with your actual file path

# Combine the two datasets
combined_data = pd.concat([year_labels_filtered, new_data_combined] , ignore_index=True)
# Displav the first few rows to verifv
#print combined data
period_counts = combined_data["Period_Label"].value_counts().sort_index()

# Map the numeric period labels back to their names for better readibility
period_names = {idx: name for name, idx in period_labels.items()}
period_counts.index = period_counts.index.map(period_names)

# Print the counts
print("Count of each period:")
print(period_counts)

            path  Period_Label
0  artwork/0.jpg           2.0
1  artwork/1.jpg           3.0
2  artwork/2.jpg           2.0
3  artwork/3.jpg           2.0
4  artwork/4.jpg           3.0
Count of each period:
Period_Label
Ancient           1543
Medieval          4844
Renaissance      19678
Baroque          11464
Enlightenment     9503
Modern            6802
Contemporary     17674
Name: count, dtype: int64


In [26]:
def clean_path(p):
    # Replace backslashes with forward slashes
    p = p.replace('\\', '/')
    # If it starts with "art500k/", remove that part
    if p.startswith("art500k/"):
        p = p[len("art500k/"):]
    return p

combined_data['path'] = combined_data['path'].apply(clean_path)
combined_data

,path,Period_Label
0,Artists1/El Greco/The Visitation##HQEQ_qXDtRrz...,3.0
1,Artists1/Gim Hongdo/Ducks In Winter##lAHeyXhD5...,4.0
2,Artists1/Jean-Baptiste-Siméon Chardin/Still Li...,4.0
3,Artists1/Utagawa Toyokuni/Sumo E Ukiyo E Print...,4.0
4,Artists1/Fra Angelico/Annunciatory Angel##TwGE...,2.0
...,...,...
71503,artwork/45570.jpg,3.0
71504,artwork/45571.jpg,5.0
71505,artwork/45572.jpg,5.0
71506,artwork/45573.jpg,5.0


In [27]:
#C:\Users\Jona\Desktop\Data Science\DS4440\Midterm\AI-Art-Detection\label_list.csv
# os.path.exists(r'C:\Users\Jona\Desktop\DataScience\DS4440\Midterm\art500k\Artists1/Margaret Bourke-White/Muncie Indiana Traffic On Walnut Street##-gG_137tF5Z2ig.jpg')

In [28]:
import os
import shutil

def copy_images_by_period(df, source_root, target_root):
    """
    df: A pandas DataFrame with columns ['path', 'Period_Label']
    source_root: Path to the folder containing the original images (art500k)
    target_root: Path to the folder where new period-based folders will be created
    """
    
    # Ensure the target root folder exists
    os.makedirs(target_root, exist_ok=True)

    for idx, row in df.iterrows():
        img_path = row['path']              # relative path inside art500k
        period_label = str(row['Period_Label'])  # ensure it's a string

        # Full source path to the image
        full_source_path = os.path.join(source_root, img_path)

        # Build the destination folder based on the period label
        dest_folder = os.path.join(target_root, period_label)
        os.makedirs(dest_folder, exist_ok=True)

        # Create the final destination path
        filename = os.path.basename(img_path)
        dest_path = os.path.join(dest_folder, filename)

        # Copy the file
        try:
            shutil.copy2(full_source_path, dest_path)
        except FileNotFoundError:
            print(f"Source file not found: {full_source_path}")
        except Exception as e:
            print(f"Error copying {full_source_path} to {dest_path}: {e}")

# Example usage:
copy_images_by_period(combined_data, r'art500k', r'art500k_period')

Source file not found: art500k/Artists1/"Best / Ever".jpg
Source file not found: art500k/Artists1/"Best / Ever".jpg
Source file not found: art500k/Artists1/"Best / Ever".jpg
Source file not found: art500k/Artists1/"Best / Ever".jpg
Error copying art500k/Artists1/Rina Banerjee/The Globe Warmed Like Egg And Once Thrice With Crack To Open She Could Pick And Pick By Hook Of Finger Eating Eaten In The Curl Of Nails One A Terrible Beauty Of Faithful Spouse She Is Yellow In All Battles With Sun When Moon Plays Like A Mouse##9QFLwDQdzSbqPA.jpg to art500k_period/6.0/The Globe Warmed Like Egg And Once Thrice With Crack To Open She Could Pick And Pick By Hook Of Finger Eating Eaten In The Curl Of Nails One A Terrible Beauty Of Faithful Spouse She Is Yellow In All Battles With Sun When Moon Plays Like A Mouse##9QFLwDQdzSbqPA.jpg: [Errno 63] File name too long: 'art500k/Artists1/Rina Banerjee/The Globe Warmed Like Egg And Once Thrice With Crack To Open She Could Pick And Pick By Hook Of Finger Eati

In [29]:
import os
from PIL import Image, UnidentifiedImageError

def remove_corrupted_images(folder_path):
    """
    Walk through the given folder (recursively) and remove any files
    that PIL cannot open as images.
    """
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            try:
                with Image.open(file_path) as img:
                    # Verify the image can be fully read
                    img.verify()
            except (UnidentifiedImageError, OSError) as e:
                # If an error occurs, it's not a valid image (or is corrupted),
                # so we remove it
                print(f"Deleting corrupted or non-image file: {file_path}")
                os.remove(file_path)
            except Exception as e:
                # Optionally catch other exceptions, log or handle them
                print(f"Unexpected error with file: {file_path}\n{e}")

# Example usage:
folder_to_check = r"art500k_period"
remove_corrupted_images(folder_to_check)

Deleting corrupted or non-image file: art500k_period/.DS_Store
Deleting corrupted or non-image file: art500k_period/2.0/Martha And Mary Magdalene##DQEEjkgqzGQdVg.jpg
Deleting corrupted or non-image file: art500k_period/6.0/Scratchpaper Rooftop Vincennes##PwEK1ciQnmnGkg.jpg
Deleting corrupted or non-image file: art500k_period/6.0/Out Of The Box##iwFybT69N9L1jQ.jpg
Deleting corrupted or non-image file: art500k_period/3.0/Hercules Feeding Diomedes To His Horses##5gHgUzKNlDFS8A.jpg
Deleting corrupted or non-image file: art500k_period/3.0/Gathering Grapes##ZgFlBm-_FQYRrQ.jpg
Deleting corrupted or non-image file: art500k_period/3.0/The Woman Taken In Adultery##tgEAvfHjznJ7mg.jpg
Deleting corrupted or non-image file: art500k_period/4.0/Two Women##iAEREs16itTapQ.jpg


In [30]:
def count_files(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files


# Example usage
directory = r'art500k_period'
file_count = count_files(directory)
print(f"Total files: {file_count}")

Total files: 49326
